add volume thresholds and only trade when the underlying isn't going exponential against the spread positions Also only run a naive backtest

In [ ]:
import nest_asyncio
nest_asyncio.apply()

import asyncio
from ib_insync import *
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import matplotlib.pyplot as plt

# --- PARAMETERS ---
durationStr = '7 D'
barSizeSetting = '5 mins'
symbol1 = 'GOOG'
lastTradeDateOrContractMonth1 = '20250725'
strike1 = 180
right1 = 'P'
symbol2 = 'GOOG'
lastTradeDateOrContractMonth2 = '20250725'
strike2 = 160
right2 = 'P'

# --- GLOBALS ---
window = 20
num_std = 2
fee = 3
initial_capital = 1000

# --- LIVE DATA FUNCTION ---
def run_live_spread():
    ib = IB()
    ib.connect('127.0.0.1', 7497, clientId=1234)

    # Define both option contracts
    c1 = Option(symbol1, lastTradeDateOrContractMonth1, strike1, right1, 'SMART', 'USD')
    c2 = Option(symbol2, lastTradeDateOrContractMonth2, strike2, right2, 'SMART', 'USD')
    ib.qualifyContracts(c1, c2)

    # Request live-updating bars
    bars1 = ib.reqHistoricalData(
        c1, endDateTime='', durationStr=durationStr, barSizeSetting=barSizeSetting,
        whatToShow='TRADES', useRTH=True, formatDate=1, keepUpToDate=True)
    bars2 = ib.reqHistoricalData(
        c2, endDateTime='', durationStr=durationStr, barSizeSetting=barSizeSetting,
        whatToShow='TRADES', useRTH=True, formatDate=1, keepUpToDate=True)

    # DataFrame to store results
    df = pd.DataFrame()

    # State for backtest
    capital = initial_capital
    position = 0
    entry_spread = None
    pending_entry = None
    pending_exit = False
    portfolio = []
    positions = []

    # --- Event handler ---
    def on_bar_update(bars, has_new_bar):
        # Sync bars by timestamp
        df1 = util.df(bars1).set_index('date').rename(columns={'close': 'Close1'})
        df2 = util.df(bars2).set_index('date').rename(columns={'close': 'Close2'})
        merged = df1[['Close1']].join(df2[['Close2']], how='inner')
        merged['spread'] = merged['Close1'] - merged['Close2']

        # Bollinger Bands
        merged['bb_middle'] = merged['spread'].rolling(window=window).mean()
        merged['bb_std'] = merged['spread'].rolling(window=window).std()
        merged['bb_upper'] = merged['bb_middle'] + num_std * merged['bb_std']
        merged['bb_lower'] = merged['bb_middle'] - num_std * merged['bb_std']

        # Signals
        merged['prev_spread'] = merged['spread'].shift(1)
        merged['prev_bb_lower'] = merged['bb_lower'].shift(1)
        merged['prev_bb_upper'] = merged['bb_upper'].shift(1)
        buy_mask = (merged['prev_spread'] > merged['prev_bb_lower']) & (merged['spread'] < merged['bb_lower'])
        sell_mask = (merged['prev_spread'] < merged['prev_bb_upper']) & (merged['spread'] > merged['bb_upper'])
        merged['bb_signal'] = 'hold'
        merged.loc[buy_mask, 'bb_signal'] = 'buy'
        merged.loc[sell_mask, 'bb_signal'] = 'sell'

        # Backtest logic (single position, delayed entry/exit)
        nonlocal capital, position, entry_spread, pending_entry, pending_exit, portfolio, positions
        portfolio = []
        positions = []
        position = 0
        entry_spread = None
        pending_entry = None
        pending_exit = False
        for i, row in merged.iterrows():
            sig = row['bb_signal']
            spread = row['spread']

            if pending_exit:
                if position == 1:
                    pnl = (spread - entry_spread) * 100
                    capital += pnl - fee
                elif position == -1:
                    pnl = (entry_spread - spread) * 100
                    capital += pnl - fee
                position = 0
                entry_spread = None
                pending_exit = False

            if pending_entry is not None and position == 0:
                if pending_entry == 'long':
                    position = 1
                    entry_spread = spread
                    capital -= fee
                elif pending_entry == 'short':
                    position = -1
                    entry_spread = spread
                    capital -= fee
                pending_entry = None

            if position == 0:
                if sig == 'buy':
                    pending_entry = 'long'
                elif sig == 'sell':
                    pending_entry = 'short'
            elif position == 1 and sig == 'sell':
                pending_exit = True
            elif position == -1 and sig == 'buy':
                pending_exit = True

            portfolio.append(capital)
            positions.append(position)

        merged['BB_Position'] = positions
        merged['BB_Portfolio'] = portfolio

        # Print latest signal and portfolio value
        if len(merged) > 0:
            print(f"Latest: {merged.index[-1]}, Spread: {merged['spread'].iloc[-1]:.2f}, "
                  f"Signal: {merged['bb_signal'].iloc[-1]}, Portfolio: {merged['BB_Portfolio'].iloc[-1]:.2f}")

        # Optional: update plotly/matplotlib plot here for live charting

    # Attach event handler
    bars1.updateEvent += on_bar_update
    bars2.updateEvent += on_bar_update

    print("Live spread monitoring started. Press Ctrl+C to exit.")
    try:
        ib.run()
    except KeyboardInterrupt:
        print("Stopped by user.")
    finally:
        ib.disconnect()

# --- Run everything ---
run_live_spread()


In [ ]:


from statsmodels.tsa.stattools import adfuller

# Assuming your DataFrame is named datafix and has a 'spread' column
spread_series = datafix['spread'].dropna()

# Run the ADF test
adf_result = adfuller(spread_series)

print('ADF Statistic: %f' % adf_result[0])
print('p-value: %f' % adf_result[1])
for key, value in adf_result[4].items():
    print('Critical Value (%s): %.3f' % (key, value))

if adf_result[1] < 0.05:
    print("Result: The spread is likely stationary (reject H0).")
else:
    print("Result: The spread is likely non-stationary (fail to reject H0).")


NameError: name 'datafix' is not defined

In [ ]:
import plotly.graph_objects as go

# --- Function to fetch latest N bars for each option ---
async def fetch_new_bars(symbol, lastTradeDateOrContractMonth, strike, right, n_bars=1):
    ib = IB()
    await ib.connectAsync('127.0.0.1', 7497, clientId=np.random.randint(1000, 10000))
    option_contract = Option(
        symbol=symbol,
        lastTradeDateOrContractMonth=lastTradeDateOrContractMonth,
        strike=strike,
        right=right,
        exchange='SMART',
        currency='USD'
    )
    await ib.qualifyContractsAsync(option_contract)
    bars = await ib.reqHistoricalDataAsync(
        contract=option_contract,
        endDateTime='',
        durationStr=f'{n_bars} min',
        barSizeSetting='1 min',
        whatToShow='TRADES',
        useRTH=True,
        formatDate=1
    )
    ib.disconnect()
    if not bars:
        print(f"No new bars returned for {symbol} {strike} {right}.")
        return pd.DataFrame()
    df = util.df(bars)
    df.set_index('date', inplace=True)
    df.rename(columns={'close': 'Close'}, inplace=True)  # Ensure 'Close' exists
    return df

# --- Function to update BB strategy with new data ---
def update_bb_strategy(datafix, datafix2, new_data1, new_data2, window=20, num_std=2):
    # Append new data if not already present
    datafix = pd.concat([datafix, new_data1[~new_data1.index.isin(datafix.index)]])
    datafix2 = pd.concat([datafix2, new_data2[~new_data2.index.isin(datafix2.index)]])
    datafix.index = datafix.index.tz_localize(None)
    datafix2.index = datafix2.index.tz_localize(None)
    # Update spread
    datafix["spread"] = datafix["Close"] - datafix2["Close"]
    # Update Bollinger Bands (only recalc for last window+1 rows for speed)
    recent = datafix.tail(window+1).copy()
    recent['bb_middle'] = recent['spread'].rolling(window=window).mean()
    recent['bb_std'] = recent['spread'].rolling(window=window).std()
    recent['bb_upper'] = recent['bb_middle'] + num_std * recent['bb_std']
    recent['bb_lower'] = recent['bb_middle'] - num_std * recent['bb_std']
    recent['prev_spread'] = recent['spread'].shift(1)
    recent['prev_bb_lower'] = recent['bb_lower'].shift(1)
    recent['prev_bb_upper'] = recent['bb_upper'].shift(1)
    buy_mask = (recent['prev_spread'] > recent['prev_bb_lower']) & (recent['spread'] < recent['bb_lower'])
    sell_mask = (recent['prev_spread'] < recent['prev_bb_upper']) & (recent['spread'] > recent['bb_upper'])
    recent['bb_signal'] = 'hold'
    recent.loc[buy_mask, 'bb_signal'] = 'buy'
    recent.loc[sell_mask, 'bb_signal'] = 'sell'
    datafix.update(recent)
    return datafix, datafix2

# --- Function to continue backtest from previous state ---
def continue_bb_backtest(datafix, initial_capital=1000, fee=1.50, multiplier=100):
    last_idx = datafix['BB_Portfolio'].last_valid_index()
    if last_idx is not None:
        capital = datafix.at[last_idx, 'BB_Portfolio']
        position = datafix.at[last_idx, 'BB_Position']
        if position == 1:
            entry_spread = datafix.loc[:last_idx][datafix['bb_signal'] == 'buy']['spread'].iloc[-1]
        else:
            entry_spread = None
        new_rows = datafix.loc[last_idx:].iloc[1:]
    else:
        capital = initial_capital
        position = 0
        entry_spread = None
        new_rows = datafix
    portfolio = []
    positions = []
    for idx, row in new_rows.iterrows():
        sig = row['bb_signal']
        spread = row['spread']
        if sig == 'buy' and position == 0:
            position = 1
            entry_spread = spread
            capital -= fee
        elif sig == 'sell' and position == 1:
            pnl = (spread - entry_spread) * multiplier
            capital += pnl
            capital -= fee
            position = 0
            entry_spread = None
        portfolio.append(capital)
        positions.append(position)
        datafix.at[idx, 'BB_Portfolio'] = capital
        datafix.at[idx, 'BB_Position'] = position
    return datafix

# --- Example usage: update with new data and plot ---
# Fetch new bars (example: 1 new bar)
new_data1 = await fetch_new_bars(symbol1, lastTradeDateOrContractMonth1, strike1, right1, n_bars=1)
new_data2 = await fetch_new_bars(symbol2, lastTradeDateOrContractMonth2, strike2, right2, n_bars=1)

# Update BB strategy DataFrame and backtest if we have new data
if not new_data1.empty and not new_data2.empty:
    datafix, datafix2 = update_bb_strategy(datafix, datafix2, new_data1, new_data2, window=20, num_std=2)
    datafix = continue_bb_backtest(datafix, initial_capital=1000, fee=1.50, multiplier=100)
    print("New bars found. Plotting latest data.")
else:
    print("No new bars found. Plotting last 30 bars of current datafix.")

# Always plot the last 30 bars (including new ones if present)
plot_df = datafix.tail(30)

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=plot_df.index, y=plot_df["spread"],
    mode='lines', name='Spread', line=dict(color='black', width=2)
))
fig.add_trace(go.Scatter(
    x=plot_df.index, y=plot_df['bb_upper'],
    mode='lines', name='BB Upper', line=dict(color='green', dash='dash')
))
fig.add_trace(go.Scatter(
    x=plot_df.index, y=plot_df['bb_lower'],
    mode='lines', name='BB Lower', line=dict(color='red', dash='dash')
))
fig.add_trace(go.Scatter(
    x=plot_df.index, y=plot_df['bb_middle'],
    mode='lines', name='BB Middle', line=dict(color='orange', dash='dot')
))
buy_signals = plot_df[plot_df["bb_signal"] == "buy"]
fig.add_trace(go.Scatter(
    x=buy_signals.index, y=buy_signals["spread"],
    mode='markers', name='Buy (on spread)',
    marker=dict(symbol='triangle-up', color='green', size=14, line=dict(color='black', width=1.5))
))
sell_signals = plot_df[plot_df["bb_signal"] == "sell"]
fig.add_trace(go.Scatter(
    x=sell_signals.index, y=sell_signals["spread"],
    mode='markers', name='Sell (on spread)',
    marker=dict(symbol='triangle-down', color='red', size=14, line=dict(color='black', width=1.5))
))
fig.update_layout(
    title="Spread, Bollinger Bands, and Buy/Sell Signals (Last 30 Bars)",
    xaxis_title="Date",
    yaxis_title="Spread",
    legend=dict(x=0.01, y=0.99),
    hovermode='x unified',
    template='plotly_white',
    width=1000,
    height=500
)
fig.show()


Error 321, reqId 4: Error validating request.-'bM' : cause - When specifying a unit, historical data request duration format is integer{SPACE}unit (S|D|W|M|Y)., contract: Option(conId=789544013, symbol='GOOG', lastTradeDateOrContractMonth='20250725', strike=180.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='GOOG  250725P00180000', tradingClass='GOOG')


No new bars returned for GOOG 180 P.


Error 321, reqId 4: Error validating request.-'bM' : cause - When specifying a unit, historical data request duration format is integer{SPACE}unit (S|D|W|M|Y)., contract: Option(conId=789543774, symbol='GOOG', lastTradeDateOrContractMonth='20250725', strike=160.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='GOOG  250725P00160000', tradingClass='GOOG')


No new bars returned for GOOG 160 P.
No new bars found. Plotting last 30 bars of current datafix.


In [ ]:
from ib_insync import *
import numpy as np
import pandas as pd

# --- Connect to IBKR ---
ib = IB()
ib.connect('127.0.0.1', 7497, clientId=np.random.randint(1000, 10000))

# --- Define option contracts ---
option1 = Option(
    symbol=symbol1,
    lastTradeDateOrContractMonth=lastTradeDateOrContractMonth1,
    strike=strike1,
    right=right1,
    exchange='SMART',
    currency='USD'
)
option2 = Option(
    symbol=symbol2,
    lastTradeDateOrContractMonth=lastTradeDateOrContractMonth2,
    strike=strike2,
    right=right2,
    exchange='SMART',
    currency='USD'
)
ib.qualifyContracts(option1, option2)

# --- Get current signal ---
last_row = datafix.iloc[-1]
signal = last_row['bb_signal']
print(f"Latest signal: {signal}")

# --- Get current positions ---
positions = {p.contract.conId: p.position for p in ib.positions()}
pos1 = positions.get(option1.conId, 0)
pos2 = positions.get(option2.conId, 0)

# Determine current spread position
# Long spread: +1 option1, -1 option2
# Short spread: -1 option1, +1 option2
if pos1 == 1 and pos2 == -1:
    current_position = 'long'
elif pos1 == -1 and pos2 == 1:
    current_position = 'short'
else:
    current_position = 'flat'

print(f"Current spread position: {current_position}")

# --- Core Logic: Exit first, then wait for next bar to enter ---
# Simulate persistent state (could be stored externally in live systems)
pending_entry = None

# 1. Exit if signal says to reverse
if current_position == 'long' and signal == 'sell':
    print("Exiting LONG spread...")
    ib.placeOrder(option1, MarketOrder('SELL', 1))  # Close long
    ib.placeOrder(option2, MarketOrder('BUY', 1))   # Close short
    pending_entry = 'short'

elif current_position == 'short' and signal == 'buy':
    print("Exiting SHORT spread...")
    ib.placeOrder(option1, MarketOrder('BUY', 1))   # Close short
    ib.placeOrder(option2, MarketOrder('SELL', 1))  # Close long
    pending_entry = 'long'

# 2. If flat and signal is actionable, enter position
elif current_position == 'flat':
    if signal == 'buy':
        print("Entering LONG spread: Buy option1, Sell option2")
        ib.placeOrder(option1, MarketOrder('BUY', 1))
        ib.placeOrder(option2, MarketOrder('SELL', 1))
    elif signal == 'sell':
        print("Entering SHORT spread: Sell option1, Buy option2")
        ib.placeOrder(option1, MarketOrder('SELL', 1))
        ib.placeOrder(option2, MarketOrder('BUY', 1))
    else:
        print("Signal is hold. No action taken.")

else:
    print("Already in correct position or waiting for exit confirmation. No new entry.")

# --- Disconnect ---
ib.disconnect()


No actionable signal (hold). No order placed.


In [ ]:
from ib_insync import *
import numpy as np

# --- Connect to IBKR Paper Trading (Port 7497) ---
ib = IB()
ib.connect('127.0.0.1', 7497, clientId=np.random.randint(1000, 10000))

# --- Define Option Contracts Using Your Variables ---
option1 = Option(
    symbol=symbol1,
    lastTradeDateOrContractMonth=lastTradeDateOrContractMonth1,
    strike=strike1,
    right=right1,
    exchange='SMART',
    currency='USD'
)

option2 = Option(
    symbol=symbol2,
    lastTradeDateOrContractMonth=lastTradeDateOrContractMonth2,
    strike=strike2,
    right=right2,
    exchange='SMART',
    currency='USD'
)

# --- Qualify Contracts ---
ib.qualifyContracts(option1, option2)

# --- Request Live Market Data ---
ticker1 = ib.reqMktData(option1, '', False, False)
ticker2 = ib.reqMktData(option2, '', False, False)

# --- Wait for Data to Populate ---
ib.sleep(2)

# --- Print Live Quotes ---
print("Option 1 (Lower Strike):")
print(f"  Bid: {ticker1.bid}, Ask: {ticker1.ask}, Last: {ticker1.last}")

print("Option 2 (Higher Strike):")
print(f"  Bid: {ticker2.bid}, Ask: {ticker2.ask}, Last: {ticker2.last}")

# --- Cancel Market Data Subscriptions ---
ib.cancelMktData(ticker1)
ib.cancelMktData(ticker2)

# --- Disconnect ---
ib.disconnect()


Error 10091, reqId 5: Part of requested market data requires additional subscription for API. See link in 'Market Data Connections' dialog for more details.GOOG NASDAQ.NMS/TOP/ALL, contract: Option(conId=789544013, symbol='GOOG', lastTradeDateOrContractMonth='20250725', strike=180.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='GOOG  250725P00180000', tradingClass='GOOG')
Error 10091, reqId 6: Part of requested market data requires additional subscription for API. See link in 'Market Data Connections' dialog for more details.GOOG NASDAQ.NMS/TOP/ALL, contract: Option(conId=789543774, symbol='GOOG', lastTradeDateOrContractMonth='20250725', strike=160.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='GOOG  250725P00160000', tradingClass='GOOG')
Error 354, reqId 5: Requested market data is not subscribed. Check API status by selecting the Account menu then under Management choose Market Data Subscription Manager and/or availabilit

Option 1 (Lower Strike):
  Bid: nan, Ask: nan, Last: nan
Option 2 (Higher Strike):
  Bid: nan, Ask: nan, Last: nan
